In [30]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.volume import ForceIndexIndicator
from ta.volume import EaseOfMovementIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.volume import NegativeVolumeIndexIndicator
from ta.trend import SMAIndicator
from ta.trend import DPOIndicator
from ta.volatility import BollingerBands
from ta.volatility import KeltnerChannel
from ta.volatility import AverageTrueRange
from ta.volatility import DonchianChannel
from ta.others import DailyLogReturnIndicator
from ta.momentum import ROCIndicator

payload=np.array(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].iloc[:,0])
payload=payload[payload!='BRK.B']
payload=payload[payload!='BF.B']
#print(payload)
boom=pd.DataFrame()

class algo:
    start="2010-01-01"
    end=['2022-01-01','2021-01-01','2020-01-01','2019-01-01','2018-01-01','2017-01-01','2016-01-01','2015-01-01','2014-01-01']
    interval='1mo'
    thr2=98
    
    def __init__(self,stock):
        self.stock=stock
        
    # get raw stock data       
    def getdata(self):
        df=(yf.download(self.stock,algo.start,algo.end[0],interval=algo.interval).dropna())
        df['log_current']=np.log(df.Close/df.Open)
        df['stock']=self.stock
        #df['Date'] = pd.to_datetime(df.index)
        df['Year'] = df.index.strftime('%Y-%m-%d').str[:4]
        #output = stock data from start to end    
        return(df)

    # apply indicators (offset 1)
    def indicators(self):
        df=self.stock
        df['log_past']=DailyLogReturnIndicator(close=df.Close.shift(1)).daily_log_return()
        df=df.dropna()
        
        #output = stock data with indicators
        return(df)
    
    # calculate percentiles based on the hustorical period (cutoff)
    def percentiles(self):
        df=self.stock
        dic={}

        for t in range (1,len(algo.end)):
            time=algo.end[t]
            df=(df[df.index<time])
            dic[time]=np.percentile(abs(df.log_past),algo.thr2)
            
        #output = dictionary: year - percentile of preveious year
        return(dic)
    
    # apply trade conditions
    def trade(self,percs):
        df=self.stock
        dff=pd.DataFrame()
        for t in range (1,len(algo.end)):
            origin=algo.end[t]
            end=algo.end[t-1]
            #print(origin,end)
            dft=(df[(df.index>=origin) & (df.index <end)])
           
            dft['trade_pass']=np.select([(dft.log_past<-percs[origin]),(dft.log_past>percs[origin])],[1,-1],default=0)
            #print(dft)
            dff=pd.concat([dff,dft])

        return(dff)
    
l1=0
l2=500

for i in range (l1,l2):
    try:
        rawdata=algo(payload[i]).getdata()
        ta=algo(rawdata).indicators()
        percs=algo(ta).percentiles()
        #print(percs)
        trades=algo(ta).trade(percs)
        #print(trades)
        boom=pd.concat([boom,trades])
    except Exception as e:
        print(e)
        pass

boom=boom[boom.Year=='2021']
#print(boom)
long=boom[boom['trade_pass']==1]
short=boom[boom['trade_pass']==-1]        

group=([boom,long,short])
#print(boom.shape,long.shape,short.shape)
for i in range(0,3): 
    trade_count=group[i].shape[0]
    win_rate=(((group[i][group[i]['log_current']>0]).shape[0]))/trade_count
    avg_per_trade=np.average(group[i].log_current)/trade_count
    minimum=np.min(group[i].log_current)
    print(trade_count,win_rate,avg_per_trade,minimum)
    
short.to_csv('short_trades.csv')
long.to_csv('long_trades.csv')
#trade_count=boom.shape[1]
#win_rate=(((boom[boom['log_current']>0]).shape[1]))/trade_count
#avg_per_trade=np.average(boom.log_current)/trade_count
#minimum=np.min(boom.log_current)
#print(trade_count,win_rate,avg_per_trade,minimum)
#print(np.nan_to_num(temp.reshape(i+1-k-l1,5),nan=0))
#dfout=pd.DataFrame(temp.reshape(i+1-k-l1,5))
#dfout.loc[dfout[3] == 'nan', 3] = 0
#print(dfout)
#dfout.to_csv('2021_bt.csv')
#boom.to_csv('boomer_trades.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [17]:
print(boom)


                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2021-10-01  176.250000  183.820007  173.779999  178.679993  177.237610   
2021-10-01  118.199997  129.399994  114.400002  128.889999  128.394272   
2021-10-01  108.250000  114.919998  106.860001  114.669998  113.296448   
2021-02-01  246.100006  261.609985  243.970001  250.899994  248.683517   
2021-10-01  323.369995  359.619995  316.010010  358.790009  357.727966   
2021-08-01   84.029999   86.389999   77.809998   82.370003   82.370003   
2021-02-01  462.279999  506.510010  450.119995  459.670013  459.670013   
2021-10-01  577.520020  659.190002  552.140015  650.359985  650.359985   
2021-06-01   25.570000   27.000000   24.629999   26.070000   25.752127   
2021-08-01   23.920000   25.480000   23.450001   23.870001   23.726572   

                 Volume   dlr_day stock        dlr  trade_pass  
Date                                          